In [1]:
def read_files(path):
    with open(path) as f:
        str_text=f.read()
    return str_text

In [3]:
#read_files('moby_dick_four_chapters.txt')

In [4]:
import spacy

In [5]:
nlp=spacy.load('en',disable=['parser','tagger','ner'])

In [6]:
nlp.max_length=1198623

In [7]:
def seperate_punc(doc_text):
    return [token.text.lower() for token in nlp(doc_text) if token.text not in '\n\n \n\n\n!"-#$%&()--.*+,-/:;<=>?@[\\]^_`{|}~\t\n ']

In [8]:
d=read_files('moby_dick_four_chapters.txt')

In [9]:
tokens=seperate_punc(d)

In [11]:
len(tokens)

11338

In [12]:
#pass 25 words--->  and let our neural network predict 26th word

In [14]:
train_len = 25 +1
text_sequences=[]
for i in range(train_len,len(tokens)):
    seq=tokens[i-train_len:i]
    text_sequences.append(seq)

In [16]:
'  '.join(text_sequences[0])

'call  me  ishmael  some  years  ago  never  mind  how  long  precisely  having  little  or  no  money  in  my  purse  and  nothing  particular  to  interest  me  on'

In [17]:
'  '.join(text_sequences[1])

'me  ishmael  some  years  ago  never  mind  how  long  precisely  having  little  or  no  money  in  my  purse  and  nothing  particular  to  interest  me  on  shore'

In [18]:
'  '.join(text_sequences[2])

'ishmael  some  years  ago  never  mind  how  long  precisely  having  little  or  no  money  in  my  purse  and  nothing  particular  to  interest  me  on  shore  i'

In [20]:
len(text_sequences)

11312

## Converting sequences to numbers

In [22]:
from keras.preprocessing.text import Tokenizer

In [24]:
tokenizer = Tokenizer()
tokenizer.fit_on_texts(text_sequences)

In [26]:
sequences=tokenizer.texts_to_sequences(text_sequences)

In [30]:
sequences[10]

[954,
 260,
 50,
 43,
 38,
 315,
 7,
 23,
 546,
 3,
 150,
 259,
 6,
 2712,
 14,
 24,
 957,
 5,
 60,
 5,
 56,
 316,
 37,
 2,
 50,
 3]

In [33]:
#tokenizer.index_word
#to view the id for each word

In [35]:
for i in sequences[0]:
    print(f"{i} : {tokenizer.index_word[i]}")

956 : call
14 : me
263 : ishmael
51 : some
261 : years
408 : ago
87 : never
219 : mind
129 : how
111 : long
954 : precisely
260 : having
50 : little
43 : or
38 : no
315 : money
7 : in
23 : my
546 : purse
3 : and
150 : nothing
259 : particular
6 : to
2712 : interest
14 : me
24 : on


In [38]:
#tokenizer.word_counts

In [39]:
vocabulary_size = len(tokenizer.word_counts)

In [40]:
vocabulary_size

2717

In [41]:
type(sequences)

list

In [42]:
import numpy as np

In [43]:
sequences=np.array(sequences)

In [44]:
sequences

array([[ 956,   14,  263, ..., 2712,   14,   24],
       [  14,  263,   51, ...,   14,   24,  957],
       [ 263,   51,  261, ...,   24,  957,    5],
       ...,
       [ 952,   12,  166, ...,  262,   53,    2],
       [  12,  166, 2711, ...,   53,    2, 2717],
       [ 166, 2711,    3, ...,    2, 2717,   26]])

In [45]:
from keras.utils import to_categorical

In [47]:
# splitting the last column as a label

In [49]:
X=sequences[:,:-1]

In [50]:
y=sequences[:,-1]

In [51]:
y=to_categorical(y,num_classes=vocabulary_size+1)

In [52]:
seq_len=X.shape[1]

In [56]:
X.shape

(11312, 25)

In [54]:
seq_len

25

In [55]:
from keras.models import Sequential
from keras.layers import Dense,LSTM,Embedding

In [59]:
def create_model(vocabulary_size,seq_len):
    model=Sequential()
    model.add(Embedding(vocabulary_size,seq_len,input_length=seq_len))
    model.add(LSTM(50,return_sequences=True))
    model.add(LSTM(50))
    model.add(Dense(50,activation='relu'))
    model.add(Dense(vocabulary_size,activation='softmax'))
    model.compile(loss='categorical_crossentropy',optimizer='adam',metrics=['accuracy'])
    model.summary()
    return model

In [60]:
model=create_model(vocabulary_size+1,seq_len)

W0808 12:57:54.475601 11852 deprecation_wrapper.py:119] From C:\ProgramData\Anaconda3\lib\site-packages\keras\optimizers.py:790: The name tf.train.Optimizer is deprecated. Please use tf.compat.v1.train.Optimizer instead.

W0808 12:57:54.518534 11852 deprecation_wrapper.py:119] From C:\ProgramData\Anaconda3\lib\site-packages\keras\backend\tensorflow_backend.py:3295: The name tf.log is deprecated. Please use tf.math.log instead.



_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_2 (Embedding)      (None, 25, 25)            67950     
_________________________________________________________________
lstm_3 (LSTM)                (None, 25, 50)            15200     
_________________________________________________________________
lstm_4 (LSTM)                (None, 50)                20200     
_________________________________________________________________
dense_1 (Dense)              (None, 50)                2550      
_________________________________________________________________
dense_2 (Dense)              (None, 2718)              138618    
Total params: 244,518
Trainable params: 244,518
Non-trainable params: 0
_________________________________________________________________


In [61]:
from pickle import dump,load

In [63]:
model.fit(X,y,batch_size=128,epochs=100,verbose=1)

Epoch 1/100
11312/11312 [==============================] - 8s 683us/step - loss: 6.1311 - acc: 0.0529
Epoch 2/100
11312/11312 [==============================] - 8s 664us/step - loss: 6.0783 - acc: 0.0530
Epoch 3/100
11312/11312 [==============================] - 7s 642us/step - loss: 6.0328 - acc: 0.0580
Epoch 4/100
11312/11312 [==============================] - 7s 653us/step - loss: 5.9736 - acc: 0.0666
Epoch 5/100
11312/11312 [==============================] - 7s 647us/step - loss: 5.8907 - acc: 0.0682
Epoch 6/100
11312/11312 [==============================] - 6s 507us/step - loss: 5.8130 - acc: 0.0692
Epoch 7/100
11312/11312 [==============================] - 5s 480us/step - loss: 5.7506 - acc: 0.0728
Epoch 8/100
11312/11312 [==============================] - 5s 474us/step - loss: 5.7018 - acc: 0.0734
Epoch 9/100
11312/11312 [==============================] - 5s 477us/step - loss: 5.6625 - acc: 0.0743
Epoch 10/100
11312/11312 [==============================] - 5s 472us/step - loss: 

In [64]:
model.save('LSTM_TEXT_GENERATION.h5')

In [66]:
dump(tokenizer,open('my_tokenizer_lstm','wb'))

## Generating new text

In [70]:
from keras.preprocessing.sequence import pad_sequences

In [73]:
def generate_text(model,tokenizer,seq_len,seed_text,num_gen_words):
    output_text=[]
    input_text=seed_text
    for i in range(num_gen_words):
        encoded_text=tokenizer.texts_to_sequences([input_text])[0]
        pad_encoded=pad_sequences([encoded_text],maxlen=seq_len,truncating='pre')
        pred_word_ind=model.predict_classes(pad_encoded,verbose=0)[0]
        pred_word=tokenizer.index_word[pred_word_ind]
        input_text+=' '+pred_word
        output_text.append(pred_word)
        
    
    
    return '  '.join(output_text)
    

In [74]:
generate_text(model,tokenizer,seq_len,text_sequences[0],5)

'shore  wriggling  tell  tell  entirely'

In [76]:
from keras.models import load_model

In [77]:
new_model=load_model('epochBIG.h5')

In [79]:
new_tokenizer=load(open('epochBIG','rb'))

In [81]:
generate_text(new_model,new_tokenizer,seq_len,text_sequences[0],10)

'raise  all  again  opinion  deadly  and  familiar  coward  deck  and'